In [1]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [12]:
# This will collect the links for all cities on craigslist. We will have to sparse through
# these and cut out the non-US cities.
def city_link_collector():
    # this is the craigslist page with every city
    main_page = requests.get('https://www.craigslist.org/about/sites')
    soup = BeautifulSoup(main_page.text, 'html.parser')
    
    all_list = []
    uscity_list = []
    city_list = []
    
    for i in range(4):
        for box in soup.find_all('div', class_='box box_{}'.format(i+1)):
            all_list.append(box.find_all('a'))
    for _ in all_list[:20:7]:
        for __ in _:
            uscity_list.append(__)
    for ___ in all_list[20][0:94]:
        uscity_list.append(___)
    
    for idx, city in enumerate(uscity_list):
        city_list.append(str(uscity_list[idx]).split('''"''')[1])
    
    '''These listings are all a subset of Miami and break the scraper if entered in this format'''
    city_list.remove('http://miami.craigslist.org/brw/')
    city_list.remove('http://miami.craigslist.org/mdc/')
    city_list.remove('http://miami.craigslist.org/pbc/')
    '''Here is the Miami list to capture the dropped entries and will not break the scraper'''
    city_list.append('http://miami.craigslist.org/')
    return city_list
    #                     #posting date
    #                     #grab the datetime element 0 for date and 1 for time
    #                     post_datetime = box.find('time', class_= 'result-date')['datetime']
    #                     post_timing.append(post_datetime)

            

In [26]:
city_list = city_link_collector()
len(city_list)

415

In [16]:
def craigslist_motorcycle_scraper(city_list):

    title = []
    price = []
    neighborhood = []
    attributes = []
    description = []
    city_names = []

    for city in city_list: 

        #get the first page of the Austin motorcycle prices
        response = requests.get('{}search/mca?s=0&bundleDuplicates=1'.format(city))
        #parse through it and make it readable
        html_soup = BeautifulSoup(response.text, 'html.parser')
        #find the total number of posts to find the limit for each page
        results_num = html_soup.find('div', class_= 'search-legend')
        #pulled the total count of posts as the upper bound of the pages array
        results_total = int(results_num.find('span', class_='totalcount').text) 
        #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
        pages = np.arange(0, results_total+1, 120)
        print("{} Posts = {}".format(city, results_total))
        print("{} Pages = {}".format(city.title(), len(pages)))

        iterations = 0

        for page in pages:         
            title_ = []
            price_ = []
            neighborhood_ = []
            attributes_ = []
            description_ = []
            city_names_ = []
            #get request      
            response = requests.get("{}search/mca?".format(city) 
                           + "s=" #the parameter for defining the page number 
                           + str(page) #the page number in the pages array from earlier
                           + "&bundleDuplicates=1")

            time.sleep(random.randint(2,3))

            #throw warning for status codes that are not 200
            if response.status_code != 200:
                warn('Request: {}; Status code: {}'.format(requests, response.status_code))

            #define the html text
            post_soup = BeautifulSoup(response.text, 'html.parser')

            #define the posts
    #         posts = page_html.find_all('li', class_= 'result-row')

            #extract data by item
    #         for post in posts:
            count = 0
            for post in post_soup.find_all('a', class_ = 'result-title hdrlnk'):
                link = post['href']
                sub_post = requests.get(link)
                sub_soup = BeautifulSoup(sub_post.text, 'html.parser')

    #             if tag.find('small') is not None:

                post_title = sub_soup.find('span', id = 'titletextonly')
                title_.append(post_title)

                post_price = sub_soup.find('span', class_ = 'price')
                price_.append(post_price)

                post_neighborhood = sub_soup.find('small')
                neighborhood_.append(post_neighborhood)

                post_attributes = sub_soup.find_all('p', attrs = {'class': 'attrgroup'})
                attributes_.append(post_attributes)

                post_description = sub_soup.find('section', id = 'postingbody')
                description_.append(post_description)

                city_names_.append(city)

                time.sleep(random.randint(2,3))
                count += 1
                
                if count == results_total:
                    break

            iterations += 1
            print("{} Page ".format(city.title()) + str(iterations) + " of {} pages".format(len(pages)) + " scraped successfully!")

            title.append(title_)
            price.append(price_)
            neighborhood.append(neighborhood_)
            attributes.append(attributes_)
            description.append(description_)
            city_names.append(city_names_)


        print("\n")
        print("{} complete!".format(city.title()))
        print(str(len(title_)) + " rows collected.")
        print("\n")
    return title, price, neighborhood, attributes, description, city_names

In [28]:
title = []

for _ in title15:
    for __ in (_):
#         __ = str(_[idx]).split('''>''')[0]
#         __ = str(_[idx]).split('''</''')[-2]
        title.append(__)

price = []

for _ in price15:
    for __ in _:
#         __ = str(_[idx]).split('''>''')[1]
#         __ = str(_[idx]).split('''</''')[-2]
        price.append(__)

neighborhood = []

for _ in neighborhood15:
    for __ in _:
#          __ = str(_[idx]).split('''(''')[1]
#         __ = str(_[idx]).split(''')''')[-2]
        neighborhood.append(__)

attributes = []

for _ in attributes15:
    for __ in _:
        attributes.append(__)

description = []

for _ in description15:
    for __ in _:
        description.append(__)

city = []

for _ in city15:
    for __ in _:
        city.append(__)

In [3]:
df = pd.read_csv('motorcycle_toy_df')

In [4]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,price,neighborhood,attributes,description,city
0,0,0,0.0,"<span id=""titletextonly"">2016 Honda CBR300r</s...","<span class=""price"">$2900</span>",<small> (Auburn)</small>,"[<p class=""attrgroup"">\n<span><b>2016 honda cb...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
1,1,1,1.0,"<span id=""titletextonly"">BAD CREDIT, NO CREDIT...",NaN,<small> (FAYETTEVILLE)</small>,"[<p class=""attrgroup"">\n<span><b> all bikes</b...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
2,2,2,2.0,"<span id=""titletextonly"">2017 Yamaha FZ 07 ver...","<span class=""price"">$5000</span>",<small> (Near Montgomery)</small>,"[<p class=""attrgroup"">\n<span><b>2017 Yamaha F...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
3,3,3,3.0,"<span id=""titletextonly"">2010 Yellow Goldwing ...",NaN,"<small>\n (<a href=""https://www.google....","[<p class=""attrgroup"">\n<span><b>2010 Honda GL...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
4,4,4,4.0,"<span id=""titletextonly"">2006 Honda XR650L *Pr...","<span class=""price"">$3800</span>",<small> (Auburn)</small>,"[<p class=""attrgroup"">\n<span><b>2006 honda xr...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/


In [9]:
for _ in df.index:
    df.replace(df['title'][_], (str(df['title'][_]).lstrip('''<span id="titletextonly">''')).rstrip('''</span>'''), inplace=True)
    df.replace(df['price'][_], int((df['price'][_].lstrip('''<span class="price">$''')).rstrip('''</span>''')), inplace=True)
    df.replace(df['neighborhood'][_], str(df['neighborhood'][_].lstrip('''<small> (''').rstrip(''')</small>''')), inplace=True)

In [10]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,price,neighborhood,attributes,description,city
0,0,0,0,2016 Honda CBR300r,"<span class=""price"">$2900</span>",<small> (Auburn)</small>,"[<p class=""attrgroup"">\n<span><b>2016 honda cb...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
1,1,1,1,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!",,<small> (FAYETTEVILLE)</small>,"[<p class=""attrgroup"">\n<span><b> all bikes</b...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
2,2,2,2,2017 Yamaha FZ 07 very low mile,"<span class=""price"">$5000</span>",<small> (Near Montgomery)</small>,"[<p class=""attrgroup"">\n<span><b>2017 Yamaha F...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
3,3,3,3,2010 Yellow Goldwing GL1800 CSC(see pics/price...,,"<small>\n (<a href=""https://www.google....","[<p class=""attrgroup"">\n<span><b>2010 Honda GL...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
4,4,4,4,2006 Honda XR650L *Price Drop*,"<span class=""price"">$3800</span>",<small> (Auburn)</small>,"[<p class=""attrgroup"">\n<span><b>2006 honda xr...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/


In [141]:
df.head()

,Unnamed: 0,Unnamed: 0.1,title,price,neighborhood,attributes,description,city
0,0,0.0,2016 Honda CBR300r,"<span class=""price"">$2900</span>",<small> (Auburn)</small>,"[<p class=""attrgroup"">\n<span><b>2016 honda cb...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
1,1,1.0,"<span id=""titletextonly"">BAD CREDIT, NO CREDIT...",NaN,<small> (FAYETTEVILLE)</small>,"[<p class=""attrgroup"">\n<span><b> all bikes</b...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
2,2,2.0,"<span id=""titletextonly"">2017 Yamaha FZ 07 ver...","<span class=""price"">$5000</span>",<small> (Near Montgomery)</small>,"[<p class=""attrgroup"">\n<span><b>2017 Yamaha F...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
3,3,3.0,"<span id=""titletextonly"">2010 Yellow Goldwing ...",NaN,"<small>\n (<a href=""https://www.google....","[<p class=""attrgroup"">\n<span><b>2010 Honda GL...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
4,4,4.0,"<span id=""titletextonly"">2006 Honda XR650L *Pr...","<span class=""price"">$3800</span>",<small> (Auburn)</small>,"[<p class=""attrgroup"">\n<span><b>2006 honda xr...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/


In [61]:
df.replace(df['attributes'][0], str(df['attributes'][0]).strip('''[<p class="attrgroup">\n<span><b>'''), inplace=True)
df.replace(df['attributes'][0], str(df['attributes'][0]).strip('''</b></span>\n<br/>\n</p>'''), inplace=True)
df.replace(df['attributes'][0], str(df['attributes'][0]).strip('''<p class="attrgroup">\n<span>'''), inplace=True)
df.replace(df['attributes'][0], str(df['attributes'][0]).strip(''' <b>'''), inplace=True)
df.replace(df['attributes'][0], str(df['attributes'][0]).strip('''</b></span>\n<br/>\n<span>'''), inplace=True)
df['attributes'][0]

'2016 honda cbr300r</b></span>\n<br/>\n</p>, <p class="attrgroup">\n<span>condition: <b>excellent</b></span>\n<br/>\n<span>engine displacement (CC): <b>286</b></span>\n<br/>\n<span>fuel: <b>gas</b></span>\n<br/>\n<span>paint color: <b>yellow</b></span>\n<br/>\n<span>street legal</span>\n<br/>\n<span>title status: <b>clean</b></span>\n<br/>\n<span>transmission: <b>manual</b></span>\n<br/>\n<span>type: <b>sport bike</b></span>\n<br/>\n</p>]'

,Unnamed: 0,Unnamed: 0.1,title,price,neighborhood,attributes,description,city
0,0,0.0,2016 Honda CBR300r,2900,<small> (Auburn)</small>,"[<p class=""attrgroup"">\n<span><b>2016 honda cb...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
1,1,1.0,"<span id=""titletextonly"">BAD CREDIT, NO CREDIT...",NaN,<small> (FAYETTEVILLE)</small>,"[<p class=""attrgroup"">\n<span><b> all bikes</b...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
2,2,2.0,"<span id=""titletextonly"">2017 Yamaha FZ 07 ver...","<span class=""price"">$5000</span>",<small> (Near Montgomery)</small>,"[<p class=""attrgroup"">\n<span><b>2017 Yamaha F...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
3,3,3.0,"<span id=""titletextonly"">2010 Yellow Goldwing ...",NaN,"<small>\n (<a href=""https://www.google....","[<p class=""attrgroup"">\n<span><b>2010 Honda GL...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
4,4,4.0,"<span id=""titletextonly"">2006 Honda XR650L *Pr...","<span class=""price"">$3800</span>",<small> (Auburn)</small>,"[<p class=""attrgroup"">\n<span><b>2006 honda xr...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
...,...,...,...,...,...,...,...,...
14098,2373,NaN,"<span id=""titletextonly"">2017 Harley-Davidson®...","<span class=""price"">$11999</span>","<small> (Peoria, AZ)</small>","[<p class=""attrgroup"">\n<span><b>2017 Harley-D...","<section id=""postingbody"">\n<div class=""print-...",https://phoenix.craigslist.org/
14099,2374,NaN,"<span id=""titletextonly"">2019 Harley-Davidson®...","<span class=""price"">$17999</span>","<small> (Peoria, AZ)</small>","[<p class=""attrgroup"">\n<span><b>2019 Harley-D...","<section id=""postingbody"">\n<div class=""print-...",https://phoenix.craigslist.org/
14100,2375,NaN,"<span id=""titletextonly"">2017 Harley-Davidson®...","<span class=""price"">$19999</span>","<small> (Peoria, AZ)</small>","[<p class=""attrgroup"">\n<span><b>2017 Harley-D...","<section id=""postingbody"">\n<div class=""print-...",https://phoenix.craigslist.org/
14101,2376,NaN,"<span id=""titletextonly"">2019 Moto Guzzi V7 II...","<span class=""price"">$5995</span>",<small> (Delivered to your Doorstep starting a...,"[<p class=""attrgroup"">\n<span><b>2019 Moto Guz...","<section id=""postingbody"">\n<div class=""print-...",https://phoenix.craigslist.org/


In [ ]:
df.head()

In [16]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,price,neighborhood,attributes,description,city
0,0,0,0,2016 Honda CBR300r,"<span class=""price"">$2900</span>",Auburn,"[<p class=""attrgroup"">\n<span><b>2016 honda cb...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
1,1,1,1,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!",,<small> (FAYETTEVILLE)</small>,"[<p class=""attrgroup"">\n<span><b> all bikes</b...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
2,2,2,2,2017 Yamaha FZ 07 very low mile,"<span class=""price"">$5000</span>",<small> (Near Montgomery)</small>,"[<p class=""attrgroup"">\n<span><b>2017 Yamaha F...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
3,3,3,3,2010 Yellow Goldwing GL1800 CSC(see pics/price...,,"<small>\n (<a href=""https://www.google....","[<p class=""attrgroup"">\n<span><b>2010 Honda GL...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
4,4,4,4,2006 Honda XR650L *Price Drop*,"<span class=""price"">$3800</span>",Auburn,"[<p class=""attrgroup"">\n<span><b>2006 honda xr...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
